# Melatih model klasifikasi dengan Pembelajaran Mesin Otomatis

Ada banyak jenis algoritma pembelajaran mesin yang dapat Anda gunakan untuk melatih model, dan terkadang tidak mudah untuk menentukan algoritma yang paling efektif untuk persyaratan data dan prediksi khusus Anda. Selain itu, Anda dapat secara signifikan memengaruhi performa prediktif model dengan melakukan pra-pemrosesan data pelatihan, menggunakan teknik seperti normalisasi, imputasi fitur yang hilang, dan lainnya. Dalam pencarian Anda untuk menemukan model terbaik untuk kebutuhan Anda, Anda mungkin perlu mencoba banyak kombinasi algoritma dan transformasi pra-pemrosesan; yang membutuhkan banyak waktu dan sumber daya komputasi.

Azure Machine Learning memungkinkan Anda mengotomatiskan perbandingan model yang dilatih menggunakan algoritma dan opsi praprosem yang berbeda. Anda dapat menggunakan antarmuka visual di [Azure Machine Learning Studio](https://ml/azure.com) atau Python SDK (v2) untuk memanfaatkan kemampuan ini. Python SDK memberi Anda kontrol yang lebih besar atas pengaturan untuk pekerjaan pembelajaran mesin otomatis, tetapi antarmuka visual lebih mudah digunakan.

## Sebelum Anda memulai

Anda akan memerlukan versi terbaru paket  **azureml-ai-ml** untuk menjalankan kode di notebook ini. Jalankan sel di bawah ini untuk memverifikasi bahwa sel diinstal.

> **Catatan:**
> Jika paket **azure-ai-ml** tidak diinstal, jalankan `pip install azure-ai-ml` untuk menginstalnya.

In [ ]:
## Menyambungkan ke ruang kerja Anda

Dengan paket SDK yang diperlukan terinstal, sekarang Anda siap untuk terhubung ke ruang kerja Anda.

Untuk menyambungkan ke ruang kerja, kita memerlukan parameter pengidentifikasi - ID langganan, nama grup sumber daya, dan nama ruang kerja. Nama grup sumber daya dan nama ruang kerja sudah diisi untuk Anda. Anda hanya memerlukan ID langganan untuk menyelesaikan perintah.

Untuk menemukan parameter yang diperlukan, klik nama langganan dan ruang kerja di kanan atas Studio. Panel akan terbuka di sebelah kanan.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Salin ID langganan dan ganti **YOUR-SUBSCRIPTION-ID** dengan nilai yang Anda salin. </p>

## Siapkan data

Anda tidak perlu membuat skrip pelatihan untuk pembelajaran mesin otomatis, tetapi Anda perlu memuat data pelatihan. 

Dalam hal ini, Anda akan menggunakan himpunan data yang berisi detail pasien diabetes. 

Untuk meneruskan himpunan data sebagai input ke pekerjaan pembelajaran mesin otomatis, data harus dalam bentuk tabular dan menyertakan kolom target. Agar data ditafsirkan sebagai himpunan data tabular, himpunan data input harus **MLTable**.

Aset data MLTable telah dibuat untuk Anda selama pengaturan. Anda dapat menjelajahi aset data dengan menavigasi ke halaman **Data** . Anda akan mengambil aset data di sini dengan menentukan nama `diabetes-training-table` dan versinya `1`. 

In [ ]:
## Mengonfigurasi pekerjaan pembelajaran mesin otomatis

Sekarang, Anda siap untuk mengonfigurasi eksperimen pembelajaran mesin otomatis.

Ketika Anda menjalankan kode di bawah ini, itu akan membuat pekerjaan pembelajaran mesin otomatis yang:

- Menggunakan kluster komputasi bernama `aml-cluster`
- `Diabetic` Menetapkan sebagai kolom target
- `accuracy` Mengatur sebagai metrik utama
- Waktu habis setelah `60` menit dari total waktu pelatihan 
- Melatih maksimal `5` model
- Tidak ada model yang akan dilatih dengan `LogisticRegression` algoritma

In [ ]:
## Menjalankan tugas pembelajaran mesin otomatis

OKE, Anda siap untuk pergi. Mari kita jalankan eksperimen pembelajaran mesin otomatis.

> **Catatan**: Ini mungkin perlu waktu!

Saat pekerjaan berjalan, Anda dapat memantaunya di Studio.

In [ ]:
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import Input

# creates a dataset based on the files in the local data folder
my_training_data_input = Input(type=AssetTypes.MLTABLE, path="azureml:diabetes-training:1")

## Configure automated machine learning job

Now, you're ready to configure the automated machine learning experiment.

When you run the code below, it will create an automated machine learning job that:

- Uses the compute cluster named `aml-cluster`
- Sets `Diabetic` as the target column
- Sets `accuracy` as the primary metric
- Times out after `60` minutes of total training time 
- Trains a maximum of `5` models
- No model will be trained with the `LogisticRegression` algorithm

In [ ]:
from azure.ai.ml import automl

# configure the classification job
classification_job = automl.classification(
    compute="aml-cluster",
    experiment_name="auto-ml-class-dev",
    training_data=my_training_data_input,
    target_column_name="Diabetic",
    primary_metric="accuracy",
    n_cross_validations=5,
    enable_model_explainability=True
)

# set the limits (optional)
classification_job.set_limits(
    timeout_minutes=60, 
    trial_timeout_minutes=20, 
    max_trials=5,
    enable_early_termination=True,
)

# set the training properties (optional)
classification_job.set_training(
    blocked_training_algorithms=["LogisticRegression"], 
    enable_onnx_compatible_models=True
)

## Run an automated machine learning job

OK, you're ready to go. Let's run the automated machine learning experiment.

> **Note**: This may take some time!

In [ ]:
# Submit the AutoML job
returned_job = ml_client.jobs.create_or_update(
    classification_job
)  

# submit the job to the backend
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

While the job is running, you can monitor it in the Studio.